In [12]:
#import Libraries

from bs4 import BeautifulSoup
import csv
from selenium import webdriver


In [14]:
#starting up the webdriver

driver = webdriver.Chrome()



In [15]:
url = 'https://www.amazon.com'
driver.get(url)

In [25]:
def geturl(searchterm):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss'
    searchterm = searchterm.replace(' ', '+')
    return template.format(searchterm)

In [26]:
url = geturl('trumpet')

https://www.amazon.com/s?k=trumpet&ref=nb_sb_noss


In [36]:
#extracting the data
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [37]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [38]:
len(results)

60

In [39]:
#Prototye
item = results[0]

In [40]:
atag = item.h2.a

In [42]:
description = atag.text.strip()

In [44]:
url = 'https://www.amazon.com' + atag.get('href')

In [47]:
price_parent = item.find('span', 'a-price')

In [48]:
price = price_parent.find('span','a-offscreen').text


In [55]:
rating = item.i.text

In [57]:
review_count = item.find('span', {'class': 'a-size-base'}).text

In [61]:
product_name = item.find('span', {'class': 'a-size-base-plus a-color-base a-text-normal'}).text

In [62]:
print(product_name)

Mendini By Cecilio Bb Trumpet - Trumpets for Beginner or Advanced Student w/Case, Cloth, Oil, Gloves - Brass Musical Instruments For Kids & Adults


In [64]:
#reading all of the search results on a page

In [70]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    product_name = item.find('span', {'class': 'a-size-base-plus a-color-base a-text-normal'}).text
    
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try: 
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributError:
        rating = ' '
        review_count = ' '
    
    
    result = (description, product_name, price, rating, review_count, url)
    
    return result   


In [71]:
records = []

results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    records.append(extract_record(item))

In [77]:
records[0]

('Mendini By Cecilio Bb Trumpet - Trumpets for Beginner or Advanced Student w/Case, Cloth, Oil, Gloves - Brass Musical Instruments For Kids & Adults',
 'Mendini By Cecilio Bb Trumpet - Trumpets for Beginner or Advanced Student w/Case, Cloth, Oil, Gloves - Brass Musical Instruments For Kids & Adults',
 '$199.99',
 '4.6 out of 5 stars',
 '4,327',
 'https://www.amazon.com/sspa/click?ie=UTF8&spc=MTozOTQ3OTY3MjY5MzkyMjI5OjE2NjE2Mzk5NzE6c3BfYXRmOjIwMDA0MzM1MjA0ODA4MTo6MDo6&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&url=%2FMendini-Cecilio-MTT-L-Trumpet-Gold%2Fdp%2FB00I8QJT78%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dtrumpet%26qid%3D1661639971%26sr%3D8-1-spons%26psc%3D1%26smid%3DACPGV300K93D')

In [82]:
def geturl(searchterm):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss'
    searchterm = searchterm.replace(' ', '+')
    
    #adding next page url data
    url = template.format(searchterm)
    url += '&page={}'
    
    return url

In [3]:
#Scraping 20 pages of results


from bs4 import BeautifulSoup
import csv
from selenium import webdriver

#getting to next page
def geturl(searchterm):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss'
    searchterm = searchterm.replace(' ', '+')
    
    #adding next page url data
    url = template.format(searchterm)
    url += '&page={}'
    
    return url

def extract_record(item): 
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
        
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
    except(AttributeError):
        return
    try:
        fullrating = item.i.text
        rating = fullrating[:3]  
    except(AttributeError):
        return
    
    try:
        review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
    except(AttributeError):
        return
   
    
    
    result = (description, price, rating, review_count, url)
    
    return result


def main(searchterm):
    driver = webdriver.Chrome()
    
    records = []
    url = geturl(searchterm)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    with open('amazonresults.csv', 'w', newline ='', encoding ='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description','Price','Rating','NumberOfReviews','url', searchterm])
        writer.writerows(records)
      






In [4]:
amazonlookup = input("what do you want to look up on Amazon?")

main(amazonlookup)

print("CSV was created for ",amazonlookup)

what do you want to look up on Amazon?trumpet
CSV was created for  trumpet
